In [1]:
import os
import sys

try:
    from google.colab import drive
    IN_COLAB = True
except:
    WORKING_DIR = '.'
    IN_COLAB = False
if IN_COLAB:
    WORKING_DIR = '/content/drive/MyDrive/Colab Notebooks'
    # Mount drive in order access Google drive
    drive.mount('/content/drive',  force_remount=True)
    
# if IN_COLAB:
#     sys.path.insert(0, WORKING_DIR)
# else:
#     # The actual code is one level higher in folder depth/structure, so we're elevating this notebook.
#     sys.path.insert(0,f".{WORKING_DIR}/")

In [ ]:
!pip install requirements.txt

In [3]:
%%writefile ./configs/t2v_train_custom.yaml

TASK_TYPE: train_t2v_entrance
ENABLE: true
use_ema: false
num_workers: 6
frame_lens: [1, 16, 16, 16, 16, 32, 32, 32]
sample_fps: [1,  8,  16, 16, 16, 8,  16, 16]
resolution: [448, 256]
vit_resolution: [224, 224]
vid_dataset: {
    'type': 'VideoDataset',
    'data_list': ['data/vid_list_custom.txt', ],
    'data_dir_list': [ "~/.cache/datasets/Appimate/train", ],
    'vit_resolution': [224, 224],
    'resolution': [448, 256],
    'get_first_frame': True,
    'max_words': 1000,
}
embedder: {
    'type': 'FrozenOpenCLIPTtxtVisualEmbedder',
    'layer': 'penultimate',
    'vit_resolution': [224, 224],
    'pretrained': 'models/open_clip_pytorch_model.bin'
}
UNet: {
    'type': 'UNetSD_T2VBase',
    'in_dim': 4,
    'y_dim': 1024,
    'upper_len': 128,
    'context_dim': 1024,
    'out_dim': 4,
    'dim_mult': [1, 2, 4, 4],
    'num_heads': 8,
    'default_fps': 8,
    'head_dim': 64,
    'num_res_blocks': 2,
    'dropout': 0.1,
    'misc_dropout': 0.4,
    'temporal_attention': True,
    'temporal_attn_times': 1,
    'use_checkpoint': True,
    'use_fps_condition': False,
    'use_sim_mask': False
}
Diffusion: {
    'type': 'DiffusionDDIM',
    'schedule': 'cosine', # cosine
    'schedule_param': {
        'num_timesteps': 1000,
        'cosine_s': 0.008,
        'zero_terminal_snr': True,
    },
    'mean_type': 'v',
    'loss_type': 'mse',
    'var_type': 'fixed_small',
    'rescale_timesteps': False,
    'noise_strength': 0.1
}
batch_sizes: {
    "1": 32,
    "4": 8,
    "8": 4,
    "16": 4,
    "32": 2
}
visual_train: {
    'type': 'VisualTrainTextImageToVideo',
    'partial_keys': [
        ['y', 'fps'],
    ],
    'use_offset_noise': False,
    'guide_scale': 9.0, 
}

Pretrain: {
    'type': pretrain_specific_strategies,
    'fix_weight': False,
    'grad_scale': 0.5,
    'resume_checkpoint': 'workspace/model_bk/model_scope_0267000.pth',
    'sd_keys_path': 'data/stable_diffusion_image_key_temporal_attention_x1.json',
}

chunk_size: 4
decoder_bs: 4
lr: 0.00003

noise_strength: 0.1
# classifier-free guidance
p_zero: 0.1
guide_scale: 3.0
num_steps: 1000000

use_zero_infer: True
viz_interval: 5        # 200
save_ckp_interval: 50   # 500

# Log
log_dir: "workspace/experiments"
log_interval: 1
seed: 8888

Overwriting ./configs/t2v_train_custom.yaml


In [4]:
!python preprocess.py \
    --json_file_path /Users/luthandomaqondo/.cache/datasets/Appimate/dataset.json \
    --output_dir /Users/luthandomaqondo/.cache/datasets/Appimate/train

Data preprocessing STARTED!
Data preprocessing DONE: 34 text prompts -> 34 videos


In [2]:
!python train_net.py --cfg configs/t2v_train.yaml

Traceback (most recent call last):
  File "/Users/luthandomaqondo/Development/Python/i2vgen-xl/train_net.py", line 14, in <module>
    from tools import *
  File "/Users/luthandomaqondo/Development/Python/i2vgen-xl/tools/__init__.py", line 3, in <module>
    from .modules import *
  File "/Users/luthandomaqondo/Development/Python/i2vgen-xl/tools/modules/__init__.py", line 1, in <module>
    from .clip_embedder import FrozenOpenCLIPEmbedder
  File "/Users/luthandomaqondo/Development/Python/i2vgen-xl/tools/modules/clip_embedder.py", line 4, in <module>
    import open_clip
ModuleNotFoundError: No module named 'open_clip'
